In [1]:
import os 
root_dir = os.getcwd()
artifacts_dir = os.path.join(root_dir, "artifacts")
data_dir = os.path.join(artifacts_dir, "data_dir")

X = os.path.join(data_dir, "X.pickle")
y = os.path.join(data_dir, "y.pickle")

In [2]:
challenge_dir = os.path.join(root_dir, "Multilabel")
img_dir = os.path.join(challenge_dir, "photos")
print(os.path.exists(img_dir))

True


In [3]:
#from keras.applications.vgg16 import decode_predictions
from keras.applications.regnet import preprocess_input
import tensorflow as tf
from keras.utils import load_img
import matplotlib.pyplot as plt 
from PIL import Image 
import seaborn as sns
import pandas as pd 
import numpy as np 
import os 
from tensorflow.keras.models import Sequential
#tf.keras.utils.img_to_array 
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.layers import Dense

In [4]:
import pickle

In [5]:
#X = pickle.load(open(X, "rb"))
y = pickle.load(open(y, "rb"))

In [6]:
# train_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
#             rotation_range=40,
#             horizontal_flip=True,
#             width_shift_range=0.2,
#             height_shift_range=0.2,
#             shear_range=2,
#             zoom_range=0.2,
#         )

In [7]:
# X = train_data_gen.flow_from_directory(X)

In [8]:
# X = X/255.0

In [9]:
# X.shape[1:]

In [10]:
# model = Sequential()
# model.add(Conv2D(64), kernel_size = (3,3), input_shape  = X.shape[1:])
# model.add(Activation("relu"))
# model.add(MaxPool2D(pool_size=(2,2)))

# model.add(Conv2D(64), (3,3))
# model.add(Activation("relu"))
# model.add(MaxPool2D(pool_size = (2,2)))

# # Dense layers wants a 1D dataset 
# model.add(Flatten())
# model.add(Dense(64))

# model.add(Dense(4, activation = "softmax"))

# model.compile(loss = "categorical_crossentropy", optimizer= "adam", metrics = ['accuray'])


# model = Sequential()
# model.add(Conv2D(128, (3, 3), activation='relu', input_shape=X.shape[1:]))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))

# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dense(4, activation = "softmax"))

In [11]:
# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
# model.fit(X, y , batch_size = 32, epochs = 30, validation_split = 0.1)

In [13]:
raw_data = []

def _load_image(img_dir):
    for num, img in enumerate(os.listdir(img_dir)):
        img_path = os.path.join(img_dir, img)
        im = load_img(img_path, target_size = (224, 224,3))
        im = img_to_array(im)
        #im  = np.expand_dims(im, axis =0)
        #print(im.shape)
        im = preprocess_input(im)
        #print(im.shape)
        raw_data.append([im, [y[num][0], y[num][1], y[num][2],y[num][3]]])


    # img = image.load_img(img_path, target_size=(224, 224))
    # img = image.img_to_array(img)
    # img = np.expand_dims(img, axis=0)
    # img = preprocess_input(img)
    # return img 
    #img = image.load_img(img_path, target_size=(224, 224))



In [14]:
_load_image(img_dir)

In [15]:
X = []
y = []
for i,data in enumerate(raw_data):
    X.append(raw_data[i][0])
    y.append(raw_data[i][1])

In [16]:
y = np.array(y)
X = np.array(X)

In [17]:
a,b,c, d = [], [] , [], []
for i in y:
    a.append(i[0]), b.append(i[1]), c.append(i[2]), d.append(i[3])

In [18]:
a = np.array(a)
b = np.array(b)
c = np.array(c)
d = np.array(d)

In [31]:
a = np.asarray(a).astype('float32').reshape((-1,1))
b = np.asarray(a).astype('float32').reshape((-1,1))
c = np.asarray(a).astype('float32').reshape((-1,1))
d = np.asarray(a).astype('float32').reshape((-1,1))


In [19]:
from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model

In [64]:
resnet = tf.keras.applications.ResNet50(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape = (224,224,3),
    pooling=None,
    classes = 1
)

In [65]:
for layer in resnet.layers:
    layer.trainable=False

In [66]:
col_a = Input(shape=(224,224, 3))
col_b = Input(shape=(224,224, 3))
col_c = Input(shape=(224,224, 3))
col_d = Input(shape=(224,224, 3))


In [67]:
col_a_feature = resnet(col_a)
col_b_feature = resnet(col_b)
col_c_feature = resnet(col_c)
col_d_feature=   resnet(col_d)



In [68]:
merged_feature = layers.concatenate([col_a_feature, col_b_feature,col_c_feature, col_d_feature], axis =-1)

In [69]:
predictions = layers.Dense(1, activation='sigmoid')(merged_feature)


In [70]:
model = Model([col_a, col_b, col_c, col_d], predictions)


In [71]:
model.compile(loss = ["categorical_crossentropy", "categorical_crossentropy", "categorical_crossentropy", "categorical_crossentropy"], optimizer= "adam", )

In [57]:
resnet.compile(loss = "categorical_crossentropy", optimizer= "adam" )

In [58]:
resnet.output

<KerasTensor: shape=(None, 7, 7, 2048) dtype=float32 (created by layer 'conv5_block3_out')>

In [59]:
resnet.fit(X, a)

ValueError: in user code:

    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-p

In [73]:
model.fit([X, X,X,X], y, epochs=1)

ValueError: in user code:

    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-p

In [33]:
a

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],

In [36]:
resnet.output

<KerasTensor: shape=(None, 7, 7, 2048) dtype=float32 (created by layer 'conv5_block3_out')>

In [25]:
resnet.compile(loss=[['categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy', 'categorical_crossentropy']], optimizer='adam', metrics=['accuracy'])

In [27]:
resnet.fit(X , y, validation_split=0.2, epochs=1, batch_size=64)


ValueError: in user code:

    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-p

In [60]:
model1 = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model1.add( tf.keras.applications.ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet', input_shape=(224, 224, 3)))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model1.add(Dense(1, activation = 'sigmoid'))

# Say not to train first layer (ResNet) model as it is already trained
model1.layers[0].trainable = False

model1.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=tf.keras.metrics.BinaryAccuracy())

In [62]:
model1.fit(X, a)

c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


31/31 [==============================] - 83s 3s/step - loss: 0.0467 - binary_accuracy: 0.9805


In [92]:
model2 = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model2.add( tf.keras.applications.ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet', input_shape=(224, 224, 3)))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model2.add(Dense(1, activation = 'sigmoid'))

# Say not to train first layer (ResNet) model as it is already trained
model2.layers[0].trainable = False

model2.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam',metrics=tf.keras.metrics.BinaryAccuracy())

In [93]:
model3 = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model3.add( tf.keras.applications.ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet', input_shape=(224, 224, 3)))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model3.add(Dense(1, activation = 'sigmoid'))

# Say not to train first layer (ResNet) model as it is already trained
model3.layers[0].trainable = False

model3.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=tf.keras.metrics.BinaryAccuracy())

In [94]:
model4 = Sequential()

# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
model4.add( tf.keras.applications.ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet', input_shape=(224, 224, 3)))

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
model4.add(Dense(1, activation = 'sigmoid'))

# Say not to train first layer (ResNet) model as it is already trained
model4.layers[0].trainable = False

model4.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), optimizer='adam', metrics=tf.keras.metrics.BinaryAccuracy())

In [78]:
y

array([[1, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       ...,
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0]])

In [81]:
c.shape

(972,)

In [82]:
X.shape

(972, 224, 224, 3)

In [83]:
model1.fit(X, a , epochs=1, validation_split=0.2, batch_size=64)
model2.fit(X, b , epochs=1, validation_split=0.2, batch_size=64)
model3.fit(X, c , epochs=1, validation_split=0.2, batch_size=64)
model4.fit(X, d , epochs=1, validation_split=0.2, batch_size=64)

13/13 [==============================] - 76s 6s/step - loss: 0.1635 - accuracy: 0.9099 - val_loss: 9.8204e-04 - val_accuracy: 1.0000


In [37]:
history = model.fit(X , y, validation_split=0.2, epochs=1, batch_size=64)

25/25 [==============================] - 167s 7s/step - loss: 2.0437 - accuracy: 0.2707 - val_loss: 1.8914 - val_accuracy: 0.2931


In [84]:
im = load_img(r"Multilabel\photos\image_005.jpg", target_size = (224, 224,3))
im = img_to_array(im)
        #im  = np.expand_dims(im, axis =0)
        #print(im.shape)
im = preprocess_input(im)

In [85]:
im = im.reshape(1, im.shape[0], im.shape[1],3)

In [86]:
im.shape

(1, 224, 224, 3)

In [87]:
model1.predict(im)

1/1 [==============================] - 1s 981ms/step


array([[0.99950236]], dtype=float32)

In [88]:
li = [model1, model2, model3, model4]
a = []
for i in li:
    a.append(i.predict(im))

1/1 [==============================] - 3s 3s/step


In [98]:
a

[array([[0.99950236]], dtype=float32),
 array([[0.9988428]], dtype=float32),
 array([[0.00166525]], dtype=float32),
 array([[0.00105461]], dtype=float32)]

In [102]:
result = []
for i in a:
    if (i[0][0]=>0.5):
        result.append(1)
    else:
        result.append(0)


SyntaxError: invalid syntax (2212629649.py, line 3)

ImportError: cannot import name 'Functional' from 'tensorflow.keras.models' (c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\api\_v2\keras\models\__init__.py)

In [122]:
input_img = tf.keras.layers.Input(shape=(224, 224, 3))

In [123]:
# 1st layer as the lumpsum weights from resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
# NOTE that this layer will be set below as NOT TRAINABLE, i.e., use it as is
resnet_in =  tf.keras.applications.ResNet50(include_top = False, pooling = 'avg', weights = 'imagenet')(input_img)

In [116]:
# resnet_in.summary()

In [117]:
# resnet_in = tf.keras.layers.Input((resnet_in))

In [124]:
# model = Sequential()


# # Say not to train first layer (ResNet) model as it is already trained
# resnet_in.layers[0].trainable = False

# 2nd layer as Dense for 2-class classification, i.e., dog or cat using SoftMax activation
out_1 = tf.keras.layers.Dense(1, activation="sigmoid")
out_2 = tf.keras.layers.Dense(1, activation="sigmoid")
out_3 = tf.keras.layers.Dense(1, activation="sigmoid")
out_4 = tf.keras.layers.Dense(1, activation="sigmoid")

model = tf.keras.Model(inputs = [input_img], outputs=[out_1, out_2, out_3, out_4])

model.compile(loss=[tf.keras.losses.BinaryCrossentropy(), tf.keras.losses.BinaryCrossentropy(), tf.keras.losses.BinaryCrossentropy(), tf.keras.losses.BinaryCrossentropy()], optimizer='adam')


ValueError: Output tensors of a Functional model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: <keras.layers.core.dense.Dense object at 0x000001FB63409BE0>

In [104]:
model.summary()

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_17 (Dense)            (None, 4)                 8196      
                                                                 
Total params: 23,595,908
Trainable params: 8,196
Non-trainable params: 23,587,712
_________________________________________________________________


In [105]:
model1.fit(X, y , epochs=1, validation_split=0.2, batch_size=64)

c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\tensorflow\python\util\dispatch.py:1082: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


ValueError: in user code:

    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 890, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-packages\keras\engine\training.py", line 948, in compute_loss
        return self.compiled_loss(
    File "c:\Users\sanjeev\miniconda3\envs\deep_learning\lib\site-p

In [106]:
y

array([[1, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       ...,
       [1, 1, 0, 0],
       [1, 1, 0, 0],
       [1, 1, 0, 0]])